# Creating the Bob Initial Commitment Transaction

In this section, we'll build Bob's lightning channel initial commitment transaction from scratch using Python. While Alice creates her own commitment transaction (showing her view of the channel state), Bob also needs his own commitment transaction that represents his view. The process will be tested using Bitcoin Core in regtest mode.

## Setup

For this notebook, we'll use the basepoints derivated in the `chapter 0 - lightning node keys derivation`, and the funding transaction created in the `chapter 1 - channel funding transaction`.

In [1]:
%run "../chapter 1 - channel funding transaction/funding transaction.ipynb"

2026-02-10T22:02:51.217000Z TestFramework (INFO): PRNG seed is: 8856263821722027120
2026-02-10T22:02:51.220000Z TestFramework (INFO): Initializing test directory /tmp/bitcoin_func_test_e3vafj6k
🟢 New TestShell started. Block height: 0
Alice Per Commitment Seed 34b581ec20bf2c6cae3d4d4dcbfddc8a3727a1e9a57c55f3520e770607898c06
Bob Per Commitment Seed 89c994b3ddad4698acee71e42d8bcace48eea739caaba371eb110e77663ec56d
Alice Payment BasePoint:  025f892a06124391e2f38ce35d943cdc09f63e203330dbd9cb6113a903e0738458
Bob Payment BasePoint:  02f98efd3f2b2fbe7bd83c419f5f64f8280798b8a9175fdb77c0091bbb95c79506
To obscure commitment number 0xb433fd43a66f
Alice funding pubkey: 95b6145ce49c7adb9041e58c3fff0b3cb60344914fe2f6160e84056bce3f2210
Alice funding privkey: d2d047708eed036ee2c1b8baa031da733b46dbe9f448a5be25ab86819318d98d
Alice funding address: bcrt1pjkmpgh8yn3adhyzpukxrllct8jmqx3y3fl30v9swsszkhn3lyggqxjjkf4
Alice sweeper pubkey: 0db41f46c0ee856819d4f41b8567b29cd9d6b4d8e3ca17519196e4568e3380cc
Alice s

In [2]:
from functions.test_framework.script import Tapbranch, TapTree, TapLeaf, CScript, TaprootSignatureHash, OP_CHECKSIG, OP_CHECKSIGVERIFY, OP_CHECKSEQUENCEVERIFY, OP_DROP, OP_16, OP_1, OP_SIZE, OP_EQUALVERIFY, OP_HASH160, OP_CHECKLOCKTIMEVERIFY
from functions.test_framework.messages import CTxInWitness, ser_string
TAPSCRIPT_VER = bytes([0xc0])
# A Taproot NUMS key is a fixed secp256k1 public key with no known private key. Used as the internal key to disable key-path spending, forcing script-path-only spends.
NUMS = ECPubKey().set(bytes.fromhex("02dca094751109d0bd055d03565874e8276dd53e926b44e3bd1bb6bf4bc130a279"))

At this point, no HTLCs have been added yet, which makes the initial commitment transaction simpler. Bob needs his own commitment transaction before the channel can be considered open. This serves as a guarantee for Bob that he can enforce the channel state on-chain if needed.

### The Unsigned Transaction

### The Input

The input is the channel funding transaction.

In [3]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have just 1 input
input_count = bytes.fromhex("01")

# Convert txid and index to bytes (little endian)
txid = (bytes.fromhex(funding_channel_txid))[::-1]
funding_channel_index = 0
index = funding_channel_index.to_bytes(4, byteorder="little", signed=False)

# For the unsigned transaction we use an empty scriptSig
scriptsig = bytes.fromhex("")

# sequence: upper 8 bits are 0x80, lower 24 bits are the upper 24 bits of the obscured commitment number
# Commitment number on the opening channel 
commitment_number = 0
# obscured commitment number is result of xor operation 
commitment_number_obscured = lower48_to_obscure ^ commitment_number
# Extract the upper 24 bits of the obscured commitment number
upper_24_bits = (commitment_number_obscured >> 24) & 0xFFFFFF
# Combine the upper 8 bits (0x80) with the lower 24 bits (upper 24 of obscured number)
sequence = (0x80 << 24) | upper_24_bits
# Convert to bytes (byte, big-endian)
sequence = sequence.to_bytes(4, byteorder='big')
# Convert to little-endian 
sequence = sequence[::-1]

inputs = (
    txid
    + index
    + varint_len(scriptsig)
    + scriptsig
    + sequence
)

print("Commitment Number Osbscured", hex(commitment_number_obscured))

Commitment Number Osbscured 0xb433fd43a66f


### The Outputs

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) defines the outputs as following:

* For every offered HTLC, if it is not trimmed, add an offered HTLC output.
* For every received HTLC, if it is not trimmed, add an received HTLC output.
* If the to_local amount is greater or equal to dust_limit_satoshis, add a to_local output.
* If the to_remote amount is greater or equal to dust_limit_satoshis, add a to_remote output.
* If option_anchors applies to the commitment transaction:
    * if to_local exists or there are untrimmed HTLCs, add a to_local_anchor output
    * if to_remote exists or there are untrimmed HTLCs, add a to_remote_anchor output

Because Bob's balance is 0 (below dust limit), his commitment transaction will only have:

* to_remote_anchor_output (330 sats)
* to_remote_output (Alice's balance)

The Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md)) specifies that transaction outputs must be sorted by value, from smallest to largest. The to_remote_anchor_output appears first with 330 sats.

### Bob First Commitment Transaction Outputs

#### The Remote Anchor Output

This is the anchor output for the remote party (Alice). It follows the same structure as the local anchor but uses Alice's delayed payment pubkey.

```
    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^  
                   |   
         +---------------+
         | P(remote) + T |
         +---------------+
                ^  
                |  
          +-----------+
          | T = t * G |
          +-----------+     
                ^                          
                |
              +---+   +---------------------------------------------------+
              | t | = | TaggedHash ("Taptweak", P(remote) || script_root) |
              +---+   +---------------------------------------------------+
                                                                  ^  
                                                                  |
                                                                +---+
                                                                | h |
                                                                +---+
                                                                  ^  
                                                                  |
                                                            +--------------+
                                                            | OP_16 OP_CSV |
                                                            +--------------+
```

#### To Remote Output

The to_remote output pays Alice (the remote party from Bob's perspective). This output uses a simple P2TR structure:
- **Key-path spend**: Alice can spend immediately using her payment key
- **No CSV delay**: Unlike to_local outputs, to_remote outputs can be spent immediately
- **Not revocable**: This output remains valid even if the commitment is revoked

```
    +------+---------------+
    | OP_1 |       Q       |
    +------+---------------+
                   ^  
                   |   
             +----------+
             | NUMS + T |
             +----------+
                   ^  
                   |  
              +-----------+
              | T = t * G |
              +-----------+     
                         ^                          
                         |
                       +---+   +----------------------------------------------+
                       | t | = | TaggedHash ("Taptweak", NUMS || script_root) |
                       +---+   +----------------------------------------------+
                                                                     ^  
                                                                     |
                                          +-------------------------------------------+
                                          | P(remote) OP_CHECKSIG OP_1 OP_CSV OP_DROP |
                                          +-------------------------------------------+
```

#### Key Derivations

For Bob's commitment transaction, we need:
- `bob_delayed_pubkey`: Bob's delayed payment key (for anchor)
- `alice_payment_pubkey`: Alice's payment key (for to_remote output)

Each commitment transaction uses unique keys derived by adding a per-commitment point to their respective base points. As defined in Basis of Lightning Technology ([BOLT 3](https://github.com/lightning/bolts/blob/master/03-transactions.md#key-derivation)):

```
pubkey = basepoint + SHA256(per_commitment_point || basepoint) * G
```

In [4]:
# Create Bob per-commitment
bob_per_commitment = per_commitment(bob_per_commitment_seed, commitment_number)
# Create Alice per-commitment
alice_per_commitment = per_commitment(alice_per_commitment_seed, commitment_number)

# Create Alice Payment Public Key (for remote anchor and to_remote outputs)
alice_payment_pubkey = derivate_key(alice_node_seed, family=5, channel_index=0).get_pubkey(bob_per_commitment.get_pub())
print(f"Alice Payment PubKey: {alice_payment_pubkey.get_bytes(bip340=True).hex()}")

# Outputs for Bob First Commitment Transaction
# 0x02 outputs
output_count = bytes.fromhex("02")

# REMOTE ANCHOR OUTPUT (for Alice)
# Anchor output amount
anchor_output_value_satoshis = 330
anchor_output_value = anchor_output_value_satoshis.to_bytes(8, byteorder="little", signed=False)

# OP_16 OP_CSV
script = CScript([OP_16, OP_CHECKSEQUENCEVERIFY])

# Method: ser_string(data) is a function which adds compactsize to input data.
hash_input = TAPSCRIPT_VER + ser_string(script)

# Anchor Output script_root
script_root = tagged_hash("TapLeaf", hash_input)

# Anchor Output Tagged Hash (uses Alice's payment pubkey - P(remote))
taptweak = tagged_hash("TapTweak", alice_payment_pubkey.get_bytes() + script_root)
alice_payment_pubkey_tweaked = alice_payment_pubkey.tweak_add(taptweak)
# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
remote_anchor_spk = bytes.fromhex("51") + varint_len(alice_payment_pubkey_tweaked.get_bytes()) + alice_payment_pubkey_tweaked.get_bytes()

Alice Payment PubKey: 4d20689e6c12ae9514854734655ef3c56c719bcba20c68a96f3e640607c7bae4


### Fee Calculation

The fee calculation for commitment transactions is based on the current `feerate_per_kw` and the **expected** weight of the transaction.

In [6]:
# Bob's first commitment expected weight 
# 0 to_local outputs (Bob has no balance), 1 to_remote output (now uses script path), 1 anchor
# Note: to_remote now uses NUMS + script path instead of simple key-path, so it's counted as TapOut
commitment_weight = taproot_commit_weight(NumTapOut=1, NumAnchorOut=1)
print("Bob's First Commitment Transaction Expected Weight:", commitment_weight)

# From open_channel message (1 sat/vb = 250 sat/kw)
feerate_per_kw = 250

# Calculate the fee for the first commitment transaction
commitment_fee = int(commitment_weight * feerate_per_kw / 1000)
print(f"Commitment Fee: {commitment_fee} sats")

Bob's First Commitment Transaction Expected Weight: 624
Commitment Fee: 156 sats


In [ ]:
# TO REMOTE OUTPUT (Alice's balance from Bob's perspective)
# to remote output amount = total channel value - anchor - fee
to_remote_value_sat = channel_value_sat - anchor_output_value_satoshis - commitment_fee
to_remote_value = to_remote_value_sat.to_bytes(8, byteorder="little", signed=False)

# For to_remote output, we use NUMS as internal key with a script path
# Script: P(remote_payment) OP_CHECKSIG OP_1 OP_CSV OP_DROP
# This ensures Alice must wait 1 block before spending (CSV delay)
script = CScript([alice_payment_pubkey.get_bytes(), OP_CHECKSIG, OP_1, OP_CHECKSEQUENCEVERIFY, OP_DROP])

# Compute the script_root
hash_input = TAPSCRIPT_VER + ser_string(script)
script_root = tagged_hash("TapBranch", hash_input)

# Compute TapTweak using NUMS as internal key
taptweak = tagged_hash("TapTweak", NUMS.get_bytes() + script_root)
NUMS_tweaked = NUMS.tweak_add(taptweak)

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
to_remote_spk = bytes.fromhex("51") + varint_len(NUMS_tweaked.get_bytes()) + NUMS_tweaked.get_bytes()

outputs = (
    anchor_output_value
    + varint_len(remote_anchor_spk)
    + remote_anchor_spk
    + to_remote_value
    + varint_len(to_remote_spk)
    + to_remote_spk
)

# Locktime: upper 8 bits are 0x20, lower 24 bits are the lower 24 bits of the obscured commitment number
# Extract the lower 24 bits of the obscured commitment number
lower_24_bits = commitment_number_obscured & 0xFFFFFF
# Combine the upper 8 bits (0x20) with the lower 24 bits (lower 24 of obscured number)
locktime = (0x20 << 24) | lower_24_bits
# Convert to bytes (1 byte, big-endian)
locktime = locktime.to_bytes(4, byteorder='big')
locktime = locktime[::-1]

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

# Decode the unsigned transaction to verify it looks correct
decoded = node.decoderawtransaction(unsigned_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

unsigned_tx:  02000000016030933d58e5f5861a73d2ee39e57e2c02e901e8e99b16006f544a06491dc7b90000000000fd33b480024a01000000000000225120539e55c2fb26353503f748858e5ea76b03a25ee8b2e54dd9eb0111dbd1b716695a400f000000000022512076ed966237e4d1dd91cc1a424bf21cd0f304b4d7671861fffa0c2c9b641bc7ef6fa64320
{
  "txid": "ed066d66e42bdc722c1e39297f5ea66de1b1fc64f1ed62359c69ff706ecf8490",
  "hash": "ed066d66e42bdc722c1e39297f5ea66de1b1fc64f1ed62359c69ff706ecf8490",
  "version": 2,
  "size": 137,
  "vsize": 137,
  "weight": 548,
  "locktime": 541304431,
  "vin": [
    {
      "txid": "b9c71d49064a546f00169be9e801e9022c7ee539eed2731a86f5e5583d933060",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 2159293437
    }
  ],
  "vout": [
    {
      "value": "0.00000330",
      "n": 0,
      "scriptPubKey": {
        "asm": "1 539e55c2fb26353503f748858e5ea76b03a25ee8b2e54dd9eb0111dbd1b71669",
        "desc": "rawtr(539e55c2fb26353503f748858e5ea76b03a25ee8b2e54dd9

## The sighash for the key path spend

This is the "Schnorr key spend" case: proving knowledge of the (tweaked) internal private key, with no script branch revealed. The message preimage is called msg_digest in [BIP-341](https://github.com/bitcoin/bips/blob/master/bip-0341.mediawiki).

In [8]:
index_of_this_input = bytes.fromhex("0000 0000")

# SIGHASH for key path spend
# See BIP-341 for details
sighash_epoch = bytes.fromhex("00")

# Control
hash_type = bytes.fromhex("00") # SIGHASH_DEFAULT (a new sighash type meaning implied SIGHASH_ALL)

# Transaction data
sha_prevouts = sha256(txid + index).digest()

sha_amounts = sha256(channel_value).digest()

# scriptPubKey P2TR: OP_1 (0x51) + PUSH32 (0x20) + xonly(32B)
sha_scriptpubkeys = sha256(
    varint_len(channel_spk)
    + channel_spk
).digest()

sha_sequences = sha256(sequence).digest()

sha_outputs = sha256(outputs).digest()

# Data about this input
spend_type = bytes.fromhex("00") # no annex present

sig_msg = (
    sighash_epoch
    + hash_type
    + version
    + locktime
    + sha_prevouts
    + sha_amounts
    + sha_scriptpubkeys
    + sha_sequences
    + sha_outputs
    + spend_type
    + index_of_this_input
)

tag_hash = sha256("TapSighash".encode()).digest()
sighash = sha256(tag_hash + tag_hash + sig_msg).digest()

## Signing the sighash

In [9]:
# Build participants to sort them using the same rule used in pubkeys aggregation.
participants = [
    (pubkey_alice_musig2.get_bytes(bip340=False), privkey_alice_musig2.get_bytes(), alice_per_commitment_seed),
    (pubkey_bob_musig2.get_bytes(bip340=False),   privkey_bob_musig2.get_bytes(), bob_per_commitment_seed),
]

# Reorder participants to match the sorted pubkeys
pk_to_tuple = {pk: (pk, sk, pcs) for pk, sk, pcs in participants}
participants = [pk_to_tuple[pk] for pk in sorted_pubkeys]

# Alice and Bob generates its own nonce pair (secnonce, pubnonce)
secnonces, pubnonces = [], []
for pk, sk, pcs in participants:
    # Use per-commitment nonce for Alice to get deterministic results
    if pk == pubkey_bob_musig2.get_bytes(bip340=False):
        sec, pub = nonce_per_commitment(pcs, commitment_number, sk, pk, agg_pubkey_tweaked, sighash)
    else:
        sec, pub = nonce_gen(sk, pk, agg_pubkey_tweaked, sighash, None)
    secnonces.append(sec)
    pubnonces.append(pub)

# Alice and Bob have exchanged the pubnonces, so they can aggregate them and create the session context
agg_nonce = nonce_agg(pubnonces)
session = SessionContext(agg_nonce, sorted_pubkeys, [bip86_tweak], [True], sighash)

# Alice and Bob produce their partial signatures
psigs = [sign(sec, sk, session) for sec, (_, sk, _) in zip(secnonces, participants)]

# Each side verifies the other's partial signature before proceeding
for i, psig in enumerate(psigs):
    assert partial_sig_verify(psig, pubnonces, sorted_pubkeys, [bip86_tweak], [True], sighash, i)

# Each side combines partial signatures into the final Schnorr signature
agg_sig = partial_sig_agg(psigs, session)

# Sanity check: verify with BIP340 against the *tweaked* x-only key
ok = schnorr_verify(sighash, agg_pubkey_tweaked, agg_sig)
print("aggregated Schnorr verifies?", ok)

aggregated Schnorr verifies? True


## The signed transaction

In [10]:
witness = (
    bytes.fromhex("01") # one stack item in the witness
    + varint_len(agg_sig)
    + agg_sig
)

# the final signed transaction
signed_tx = (
    version
    + marker
    + flag
    + input_count
    + inputs
    + output_count
    + outputs
    + witness
    + locktime
)

print("signed tx: ",signed_tx.hex())
# Decode the signed transaction to verify it looks correct
decoded = node.decoderawtransaction(signed_tx.hex())
print(json.dumps(decoded, indent=2, default=str))

print(node.testmempoolaccept(rawtxs=[signed_tx.hex()]))

signed tx:  020000000001016030933d58e5f5861a73d2ee39e57e2c02e901e8e99b16006f544a06491dc7b90000000000fd33b480024a01000000000000225120539e55c2fb26353503f748858e5ea76b03a25ee8b2e54dd9eb0111dbd1b716695a400f000000000022512076ed966237e4d1dd91cc1a424bf21cd0f304b4d7671861fffa0c2c9b641bc7ef0140d8f522a181854845c9ea33d8e355f3f8c168bf7cc957be52d9d8217f90849c2764747ef6438b809083fae9d3e6e16b2e6e24ada94b11455bc7b267e163fab3926fa64320
{
  "txid": "ed066d66e42bdc722c1e39297f5ea66de1b1fc64f1ed62359c69ff706ecf8490",
  "hash": "b4bc4938bf3daad65ca8fa38b4a0ec227104a78d9c110401d1aee6bf03e37798",
  "version": 2,
  "size": 205,
  "vsize": 154,
  "weight": 616,
  "locktime": 541304431,
  "vin": [
    {
      "txid": "b9c71d49064a546f00169be9e801e9022c7ee539eed2731a86f5e5583d933060",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "txinwitness": [
        "d8f522a181854845c9ea33d8e355f3f8c168bf7cc957be52d9d8217f90849c2764747ef6438b809083fae9d3e6e16b2e6e24ada94b11455bc7b

## Summary

This chapter built Bob's first commitment transaction, showing the channel state from Bob's perspective:

### Transaction Overview
- **Commitment number**: 0 (first commitment)
- **Input**: Channel funding output (0.01 BTC)
- **Sequence/Locktime**: Encode obscured commitment number for state tracking

### Output 1: Remote Anchor (330 sats)
- Allows Alice (remote party) to CPFP the commitment transaction for fee bumping
- Script path: `OP_16 OP_CHECKSEQUENCEVERIFY` (anyone can spend after 16 blocks)
- **Internal key**: Alice's payment pubkey - `P(remote)` (tweaked)

### Output 2: To_Remote (~0.01 BTC minus fees)
- Alice's channel balance from Bob's perspective
- **Internal key**: NUMS point (disables key-path spending)
- **Script path**: `P(remote) OP_CHECKSIG OP_1 OP_CSV OP_DROP`
- **1-block CSV delay**: Alice must wait 1 block before spending via script path
- **Not revocable**: This output remains valid even if the commitment is revoked

### Key Differences from Alice's Commitment

**Alice's commitment transaction** (her view):
- to_local output: Alice's balance with 144-block CSV delay and revocation path
- to_local_anchor: For Alice to CPFP, uses Alice's delayed payment pubkey

**Bob's commitment transaction** (his view):
- to_remote output: Alice's balance with 1-block CSV delay, no revocation path
- to_remote_anchor: For Alice to CPFP, uses Alice's payment pubkey
- No to_local output: Bob has 0 balance (below dust)

### Asymmetry in Lightning

Each party holds their counterparty's commitment transaction:
- **Alice holds Bob's commitment**: If Bob misbehaves, Alice broadcasts it
- **Bob holds Alice's commitment**: If Alice misbehaves, Bob broadcasts it

This asymmetry is crucial:
- Your own funds in your commitment have long CSV delays (to_local: 144 blocks)
- Counterparty's funds in your commitment have minimal CSV delay (to_remote: 1 block)
- This gives the honest party time to catch and penalize revoked states

### Anchor Output Keys
- **Local anchor**: Uses delayed payment pubkey (`P(local_delayed)`)
- **Remote anchor**: Uses payment pubkey (`P(remote)`)
- This distinction ensures proper key management across different output types

### Fee Calculation
- **Expected weight**: Calculated based on 1 TapOut (to_remote with script) + 1 Anchor
- **Fee rate**: 250 sat/kw (1 sat/vB)
- **Fee amount**: Varies based on transaction weight
- **Final to_remote value**: channel_value - anchor (330 sats) - commitment_fee

### Key Takeaway
Bob's initial commitment transaction completes the bilateral commitment exchange. Both Alice and Bob now have:
1. Their own commitment transaction (their view of channel state)
2. The ability to enforce that state on-chain if needed
3. Asymmetric security guarantees that protect the honest party

The 1-block CSV delay on to_remote outputs provides minimal delay for the counterparty while maintaining the Taproot script structure. The use of payment pubkey for remote anchors (instead of delayed payment pubkey) simplifies the key structure for counterparty-controlled outputs.

Only after both parties have valid commitment transactions should the funding transaction be broadcast to the blockchain.